In [188]:
# Drop whatever is in the database 
from datetime import datetime
import pandas as pd

env_data = pd.read_csv('../.env')
ENV = {}
for row in env_data[env_data.columns[0]]:
    #key, value = 
    row_data = row.split('=')
    key = row_data[0]
    value = '='.join(row_data[1:])
    value = value.replace("'", "").replace('"', '')
    ENV.update({key: value})

from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql

# initialize Connector object
connector = Connector()

SELECTED_DB_NAME = "COMMON_DATA"
# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        ENV['INSTANCE_CONNECTION_NAME'],
        "pymysql",
        user=ENV['DB_USER'],
        password=ENV['DB_PASS'],
        db=SELECTED_DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql:///COMMON_DATA.db",
    creator=getconn,
    echo = True
)

show_dbs_statement = sqlalchemy.text('SHOW DATABASES;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 21:55:07,936 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-12 21:55:07,938 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 21:55:08,448 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-12 21:55:08,451 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 21:55:08,709 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-12 21:55:08,711 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 21:55:09,220 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 21:55:09,221 INFO sqlalchemy.engine.Engine SHOW DATABASES;
2023-03-12 21:55:09,222 INFO sqlalchemy.engine.Engine [generated in 0.00265s] {}
2023-03-12 21:55:09,479 INFO sqlalchemy.engine.Engine ROLLBACK


,Database
0,1A
1,COMMON_DATA
2,ORDER1
3,information_schema
4,mysql
5,performance_schema
6,sys


In [150]:
getOrderDetailsSQL = sqlalchemy.text(f'SELECT * FROM orders WHERE orderId={1};')
# listDataBasesSQL = sqlalchemy.text('SHOW DATABASES;')
# connect 
with pool.connect() as db_conn:
    # 
    result = db_conn.execute(getOrderDetailsSQL)
    resultData = result.fetchall()
    resultKeys = result.keys()
    if len(resultData):
        # log += f'\nOrder #{orderId} entry exists in orders list.'
        orderExists = True
        data = dict(zip(resultKeys, list(resultData[0])))
    else:
        data = {}
        orderExists = False
        # log += f'\nOrder #{orderId} entry does not exist in orders list.'

    # result = db_conn.execute(listDataBasesSQL)
    # resultData = result.fetchall()

2023-03-12 18:59:12,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 18:59:12,908 INFO sqlalchemy.engine.Engine SELECT * FROM orders WHERE orderId=1;
2023-03-12 18:59:12,908 INFO sqlalchemy.engine.Engine [cached since 104.4s ago] {}
2023-03-12 18:59:13,167 INFO sqlalchemy.engine.Engine ROLLBACK


In [151]:
data

{}

In [55]:
orderId = 1
orderDBName = f'ORDER{orderId}'
(orderDBName,) in data

True

In [ ]:
getOrderDetails = sqlalchemy.text(f'SELECT * FROM orders WHERE orderId={orderId};')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(getOrderDetails)
    resultData = result.fetchall()
    resultKeys = result.keys()
    if len(resultData):
        resultDict = dict(zip(resultKeys, list(resultData[0])))
    else:
        resultDict = {}

In [128]:
getOrderDetails = sqlalchemy.text(f'SELECT * FROM orders;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(getOrderDetails)
    resultData = result.fetchall()
    resultKeys = result.keys()
    resultDict = dict(zip(resultKeys, zip(*resultData)))
    
pd.DataFrame(resultDict)

2023-03-12 18:49:42,080 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 18:49:42,080 INFO sqlalchemy.engine.Engine SELECT * FROM orders;
2023-03-12 18:49:42,080 INFO sqlalchemy.engine.Engine [cached since 51.73s ago] {}
2023-03-12 18:49:42,336 INFO sqlalchemy.engine.Engine ROLLBACK


""


In [103]:
ORDER_DETAILS = ['site', 'unitCode', 'timeZone']

data = {
    'orderId': 1,
    'site': 'CENTURY ENKA BHOSARI',
    'unitCode': 'CPRG160/17.5/1001',
    'timeZone': 'Asia/Kolkata'
}

{key:data[key] for key in ORDER_DETAILS if key in data}

{'site': 'CENTURY ENKA BHOSARI',
 'unitCode': 'CPRG160/17.5/1001',
 'timeZone': 'Asia/Kolkata'}

In [10]:
insert_order = sqlalchemy.text(
    "INSERT INTO orders (orderId, site, unitCode, createDate, timeZone) VALUES (:orderId, :site, :unitCode, :createDate, :timeZone) ON DUPLICATE KEY UPDATE site=:site, unitCode=:unitCode, timeZone=:timeZone;",
)

orderdata = {
    'orderId': 2,
    'site': 'KNPL',
    'unitCode': 'VTRG',
    'timeZone': 'Asia/Kolkata',
    'createDate': datetime.utcnow()
}

with pool.connect() as db_conn:
    # insert into database
    db_conn.execute(insert_order, parameters=orderdata)
    db_conn.commit()

2023-03-12 12:38:33,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 12:38:33,537 INFO sqlalchemy.engine.Engine INSERT INTO orders (orderId, site, unitCode, createDate, timeZone) VALUES (%(orderId)s, %(site)s, %(unitCode)s, %(createDate)s, %(timeZone)s) ON DUPLICATE KEY UPDATE site=%(site)s, unitCode=%(unitCode)s, timeZone=%(timeZone)s;
2023-03-12 12:38:33,538 INFO sqlalchemy.engine.Engine [cached since 44.2s ago] {'orderId': 2, 'site': 'KNPL', 'unitCode': 'VTRG', 'createDate': datetime.datetime(2023, 3, 12, 7, 8, 33, 536968), 'timeZone': 'Asia/Kolkata'}
2023-03-12 12:38:33,897 INFO sqlalchemy.engine.Engine COMMIT


In [176]:
sql_statement = sqlalchemy.text('DELETE FROM orders')
with pool.connect() as db_conn:
    # insert into database
    db_conn.execute(sql_statement)
    db_conn.commit()

sql_statement = sqlalchemy.text('DROP DATABASE ORDER1;')
with pool.connect() as db_conn:
    # insert into database
    db_conn.execute(sql_statement)
    db_conn.commit()

2023-03-12 19:38:37,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:38:37,060 INFO sqlalchemy.engine.Engine DELETE FROM orders
2023-03-12 19:38:37,061 INFO sqlalchemy.engine.Engine [generated in 0.00256s] {}
2023-03-12 19:38:37,322 INFO sqlalchemy.engine.Engine COMMIT
2023-03-12 19:38:37,837 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:38:37,838 INFO sqlalchemy.engine.Engine DROP DATABASE ORDER1;
2023-03-12 19:38:37,839 INFO sqlalchemy.engine.Engine [generated in 0.00197s] {}
2023-03-12 19:38:38,156 INFO sqlalchemy.engine.Engine COMMIT


In [185]:
import requests
response = requests.get('https://testdockerflask-mucvbrxuxq-uc.a.run.app/onboarding')

responseDict = response.json()

In [132]:
responseDict['schema']

{'orderId': '',
 'site': '',
 'unitCode': '',
 'timeZone': '',
 'metaData': [{'standardTag': 'XXXXX1_XXX_XXXXX_XXXX1',
   'description': '',
   'dataTagId': '',
   'unitId': 1},
  {'standardTag': 'XXXXX1_XXX_XXXXX_XXXX2',
   'description': '',
   'dataTagId': '',
   'unitId': 1}],
 'fixedData': [{'standardTag': 'XXXXX2_XXX_XXXXX_XXXX1',
   'description': '',
   'value': 0.0,
   'unitId': 1},
  {'standardTag': 'XXXXX2_XXX_XXXXX_XXXX2',
   'description': '',
   'value': 0.0,
   'unitId': 1}]}

In [133]:
metaDataDF = pd.read_excel('centuryEnkaMetaData-standard.xlsx')
metaDataDF.head()

,description,dataTagId,unitId,standardTag
0,Process Inlet Steam Pressure,PT2_7_SCALE_PV,2,PROSS1_INN_STEAM_PRSS1
1,Furnace Inlet Totalized Fuel,TOTALISE_FUEL_FLOW,11,FURNC1_INN_FUELS_TOTZ1
2,Process Inlet Totalized Steam,TOTAL_STEAM,11,PROSS1_INN_STEAM_TOTZ1
3,Process Outlet/Make up Water Temperature,TE1_5_SCALE_PV,7,FWTNK1_OUT_WATER_TEMP1
4,Process Inlet Exit Steam Temperature,TE2_8_SCALE_PV,7,PROSS1_INN_STEAM_TEMP1


In [134]:
metaData = metaDataDF.to_dict(orient='records')

In [135]:
fixedDataDF = pd.read_excel('centuryEnkaConstantsData.xlsx')
fixedDataDF.head()

,description,standardTag,value,unitId
0,Direct Efficiency Averaging Time Window,DREFF1_MID_RLAVG_TIME1,60.000,8
1,Gross calorific value of fuel in use,FURNC1_INN_FUELS_HGCV1,3898.000,10
2,Net calorific value of fuel in use,FURNC1_INN_FUELS_HNCV1,3620.000,10
3,Fuel profile constant term,SCOMB1_MID_FUELS_COFW1,1.002,1
4,Fuel profile coefficient of exponential term,SCOMB1_MID_FUELS_COFW2,0.981,1


In [136]:
fixedData = fixedDataDF.to_dict(orient='records')

In [137]:
orderPayload = {
    'orderId': 1,
    'site': 'CENTURY ENKA BHOSARI',
    'unitCode': 'CPRG160/17.5/1001',
    'timeZone': 'Asia/Kolkata',
    'metaData' : metaData,
    'fixedData' : fixedData,
}

# orderPayload

In [138]:
META_TABLE_COLUMNS = ['standardTag', 'description', 'dataTagId', 'unitId']
metaTableRecords = [{key: record[key] for key in META_TABLE_COLUMNS} for record in metaData]

pd.DataFrame(metaTableRecords)

,standardTag,description,dataTagId,unitId
0,PROSS1_INN_STEAM_PRSS1,Process Inlet Steam Pressure,PT2_7_SCALE_PV,2
1,FURNC1_INN_FUELS_TOTZ1,Furnace Inlet Totalized Fuel,TOTALISE_FUEL_FLOW,11
2,PROSS1_INN_STEAM_TOTZ1,Process Inlet Totalized Steam,TOTAL_STEAM,11
3,FWTNK1_OUT_WATER_TEMP1,Process Outlet/Make up Water Temperature,TE1_5_SCALE_PV,7
4,PROSS1_INN_STEAM_TEMP1,Process Inlet Exit Steam Temperature,TE2_8_SCALE_PV,7
5,CTRLX1_MID_XXXXX_LOAD1,System Load,BOILER_LOAD,1
6,BOILR1_INN_WATER_FLOW1,Feed water to boiler -- mass flow rate,FEED_WATER_FLOW,3
7,BOILR1_INN_WATER_TEMP1,Feed water to boiler -- temperature,TE1_5_SCALE_PV,7
8,BOILR1_MID_DRUMX_PRSS1,Boiler tank pressure,PT2_7_SCALE_PV,2
9,BOILR1_OUT_STEAM_FLOW1,Boiler drum steam-out mass flow rate,FT2_14A_SCALE_PV,3


In [187]:
import requests
response = requests.post('https://testdockerflask-mucvbrxuxq-uc.a.run.app/onboarding', json=orderPayload)
responseDict = response.json()

print(responseDict['log'])


Order #1 entry pre-exists in the orders list.
Database ORDER1 already exists.
Order details have been altered as per request
Table found: META_DATA.
Table found: FIXED_DATA.


In [178]:
show_dbs_statement = sqlalchemy.text('SELECT * FROM orders;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 19:39:01,385 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:39:01,386 INFO sqlalchemy.engine.Engine SELECT * FROM orders;
2023-03-12 19:39:01,387 INFO sqlalchemy.engine.Engine [generated in 0.00165s] {}
2023-03-12 19:39:01,643 INFO sqlalchemy.engine.Engine ROLLBACK


,orderId,site,unitCode,createDate,timeZone
0,1,CENTURY ENKA BHOSARI,CPRG160/17.5/1001,2023-03-12 14:08:55,Asia/Kolkata


In [189]:
connector = Connector()

SELECTED_DB_NAME = "ORDER1"
# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        ENV['INSTANCE_CONNECTION_NAME'],
        "pymysql",
        user=ENV['DB_USER'],
        password=ENV['DB_PASS'],
        db=SELECTED_DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql:///ORDER1.db",
    creator=getconn,
    echo = True
)

show_dbs_statement = sqlalchemy.text('SHOW DATABASES;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 22:23:50,416 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-03-12 22:23:50,416 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 22:23:50,943 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-03-12 22:23:50,944 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 22:23:51,201 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-03-12 22:23:51,203 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 22:23:51,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 22:23:51,721 INFO sqlalchemy.engine.Engine SHOW DATABASES;
2023-03-12 22:23:51,722 INFO sqlalchemy.engine.Engine [generated in 0.00191s] {}
2023-03-12 22:23:51,977 INFO sqlalchemy.engine.Engine ROLLBACK


,Database
0,1A
1,COMMON_DATA
2,ORDER1
3,information_schema
4,mysql
5,performance_schema
6,sys


In [191]:
show_dbs_statement = sqlalchemy.text('SELECT * FROM TEMP_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 22:51:18,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 22:51:18,396 INFO sqlalchemy.engine.Engine SELECT * FROM TEMP_DATA;
2023-03-12 22:51:18,396 INFO sqlalchemy.engine.Engine [cached since 1624s ago] {}
2023-03-12 22:51:18,654 INFO sqlalchemy.engine.Engine ROLLBACK


,t,PROSS1_INN_STEAM_PRSS1,FURNC1_INN_FUELS_TOTZ1,PROSS1_INN_STEAM_TOTZ1,FWTNK1_OUT_WATER_TEMP1,PROSS1_INN_STEAM_TEMP1,CTRLX1_MID_XXXXX_LOAD1,BOILR1_INN_WATER_FLOW1,BOILR1_INN_WATER_TEMP1,BOILR1_MID_DRUMX_PRSS1,BOILR1_OUT_STEAM_FLOW1,BOILR1_MID_DRUMX_LEVL1
0,2023-03-12 16:36:57,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,None,None
1,2023-03-12 17:20:56,1332723.735,47432050.0,424826470.0,376.15,None,None,None,376.15,1332723.735,None,None


In [180]:
show_dbs_statement = sqlalchemy.text('SHOW TABLES FROM ORDER1;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)
    data = result.fetchall()

list(zip(*data))[0]

2023-03-12 19:39:34,945 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:39:34,946 INFO sqlalchemy.engine.Engine SHOW TABLES FROM ORDER1;
2023-03-12 19:39:34,946 INFO sqlalchemy.engine.Engine [generated in 0.00118s] {}
2023-03-12 19:39:35,201 INFO sqlalchemy.engine.Engine ROLLBACK


('FIXED_DATA', 'META_DATA', 'TEMP_DATA')

In [68]:
meta = sqlalchemy.MetaData()
meta.reflect(bind=pool)

2023-03-12 16:24:09,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 16:24:09,937 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `ORDER1`
2023-03-12 16:24:09,937 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 16:24:10,196 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `FIXED_DATA`
2023-03-12 16:24:10,197 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 16:24:10,456 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `META_DATA`
2023-03-12 16:24:10,458 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 16:24:10,715 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `TEMP_DATA`
2023-03-12 16:24:10,717 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-12 16:24:11,037 INFO sqlalchemy.engine.Engine ROLLBACK


In [79]:
list(meta.tables)

['FIXED_DATA', 'META_DATA', 'TEMP_DATA']

In [76]:
[col.name for col in meta.tables['META_DATA'].columns]

['standardTag', 'description', 'dataTagId', 'unitId']

In [77]:
[col.name for col in meta.tables['FIXED_DATA'].columns]

['standardTag', 'description', 'value', 'unitId']

In [78]:
[col.name for col in meta.tables['TEMP_DATA'].columns]

['t',
 'PROSS1_INN_STEAM_PRSS1',
 'FURNC1_INN_FUELS_TOTZ1',
 'PROSS1_INN_STEAM_TOTZ1',
 'FWTNK1_OUT_WATER_TEMP1',
 'PROSS1_INN_STEAM_TEMP1',
 'CTRLX1_MID_XXXXX_LOAD1',
 'BOILR1_INN_WATER_FLOW1',
 'BOILR1_INN_WATER_TEMP1',
 'BOILR1_MID_DRUMX_PRSS1',
 'BOILR1_OUT_STEAM_FLOW1',
 'BOILR1_MID_DRUMX_LEVL1']

In [169]:
show_dbs_statement = sqlalchemy.text('DESC TEMP_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 19:34:55,597 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:34:55,598 INFO sqlalchemy.engine.Engine DESC TEMP_DATA;
2023-03-12 19:34:55,599 INFO sqlalchemy.engine.Engine [generated in 0.00207s] {}
2023-03-12 19:34:55,957 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,t,datetime,NO,PRI,None,
1,PROSS1_INN_STEAM_PRSS1,double,YES,,None,
2,FURNC1_INN_FUELS_TOTZ1,double,YES,,None,
3,PROSS1_INN_STEAM_TOTZ1,double,YES,,None,
4,FWTNK1_OUT_WATER_TEMP1,double,YES,,None,
5,PROSS1_INN_STEAM_TEMP1,double,YES,,None,
6,CTRLX1_MID_XXXXX_LOAD1,double,YES,,None,
7,BOILR1_INN_WATER_FLOW1,double,YES,,None,
8,BOILR1_INN_WATER_TEMP1,double,YES,,None,
9,BOILR1_MID_DRUMX_PRSS1,double,YES,,None,


In [21]:
show_dbs_statement = sqlalchemy.text('DESC FIXED_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 01:33:43,812 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 01:33:43,813 INFO sqlalchemy.engine.Engine DESC FIXED_DATA;
2023-03-12 01:33:43,815 INFO sqlalchemy.engine.Engine [generated in 0.00318s] {}
2023-03-12 01:33:44,080 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,standardTag,varchar(30),NO,PRI,None,
1,description,text,YES,,None,
2,value,double,YES,,None,
3,unitId,int,YES,,None,


In [22]:
show_dbs_statement = sqlalchemy.text('DESC TEMP_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 01:34:04,956 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 01:34:04,958 INFO sqlalchemy.engine.Engine DESC TEMP_DATA;
2023-03-12 01:34:04,960 INFO sqlalchemy.engine.Engine [generated in 0.00316s] {}
2023-03-12 01:34:05,523 INFO sqlalchemy.engine.Engine ROLLBACK


,Field,Type,Null,Key,Default,Extra
0,t,datetime,NO,PRI,None,
1,PROSS1_INN_STEAM_PRSS1,double,YES,,None,
2,FURNC1_INN_FUELS_TOTZ1,double,YES,,None,
3,PROSS1_INN_STEAM_TOTZ1,double,YES,,None,
4,FWTNK1_OUT_WATER_TEMP1,double,YES,,None,
5,PROSS1_INN_STEAM_TEMP1,double,YES,,None,
6,CTRLX1_MID_XXXXX_LOAD1,double,YES,,None,
7,BOILR1_INN_WATER_FLOW1,double,YES,,None,
8,BOILR1_INN_WATER_TEMP1,double,YES,,None,
9,BOILR1_MID_DRUMX_PRSS1,double,YES,,None,


In [181]:
show_dbs_statement = sqlalchemy.text('SELECT * FROM META_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 19:39:45,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:39:45,522 INFO sqlalchemy.engine.Engine SELECT * FROM META_DATA;
2023-03-12 19:39:45,523 INFO sqlalchemy.engine.Engine [generated in 0.00121s] {}
2023-03-12 19:39:45,832 INFO sqlalchemy.engine.Engine ROLLBACK


,standardTag,description,dataTagId,unitId
0,BOILR1_INN_WATER_FLOW1,Feed water to boiler -- mass flow rate,FEED_WATER_FLOW,3
1,BOILR1_INN_WATER_TEMP1,Feed water to boiler -- temperature,TE1_5_SCALE_PV,7
2,BOILR1_MID_DRUMX_LEVL1,Boiler drum level transmitter output,LT2_4_SCALE_PV,1
3,BOILR1_MID_DRUMX_PRSS1,Boiler tank pressure,PT2_7_SCALE_PV,2
4,BOILR1_OUT_STEAM_FLOW1,Boiler drum steam-out mass flow rate,FT2_14A_SCALE_PV,3
5,CTRLX1_MID_XXXXX_LOAD1,System Load,BOILER_LOAD,1
6,FURNC1_INN_FUELS_TOTZ1,Furnace Inlet Totalized Fuel,TOTALISE_FUEL_FLOW,11
7,FWTNK1_OUT_WATER_TEMP1,Process Outlet/Make up Water Temperature,TE1_5_SCALE_PV,7
8,PROSS1_INN_STEAM_PRSS1,Process Inlet Steam Pressure,PT2_7_SCALE_PV,2
9,PROSS1_INN_STEAM_TEMP1,Process Inlet Exit Steam Temperature,TE2_8_SCALE_PV,7


In [184]:
show_dbs_statement = sqlalchemy.text('SELECT * FROM FIXED_DATA;')
with pool.connect() as db_conn:
    # insert into database
    result = db_conn.execute(show_dbs_statement)

    keys = result.keys()
    data = result.fetchall()

    resultDict = dict(zip(keys, zip(*data)))

pd.DataFrame(resultDict)

2023-03-12 19:49:44,878 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:49:44,879 INFO sqlalchemy.engine.Engine SELECT * FROM FIXED_DATA;
2023-03-12 19:49:44,880 INFO sqlalchemy.engine.Engine [cached since 595.5s ago] {}
2023-03-12 19:49:45,134 INFO sqlalchemy.engine.Engine ROLLBACK


,standardTag,description,value,unitId
0,DREFF1_MID_RLAVG_TIME1,Direct Efficiency Averaging Time Window,60.000000,8
1,FURNC1_INN_FUELS_HGCV1,Gross calorific value of fuel in use,3898.000000,10
2,FURNC1_INN_FUELS_HNCV1,Net calorific value of fuel in use,3620.000000,10
3,HEATX1_MID_XXXXX_COFW4,Heat Transfer Efficiency,0.833000,1
4,HEATX1_MID_XXXXX_COFW5,Coefficiency W5 - Load vs X_F,0.416000,1
5,HEATX1_MID_XXXXX_COFW6,Coefficiency W6 - Load vs X_F,0.105000,1
6,HEATX1_MID_XXXXX_COFW7,Coefficiency W7 - Load vs X_F,0.053000,1
7,HYDRO1_MID_DNCMR_AREA1,Area of downcomer cross section,0.032283,1
8,HYDRO1_MID_DNCMR_CFRN1,dimensionless friction coefficienct Eq 16@Astr...,2333.807750,1
9,HYDRO1_MID_DNCMR_VOLM1,Volume of downcomer,0.326800,1


In [119]:
# insertMetaDataStatement = sqlalchemy.text('INSERT INTO META_DATA (standardTag, description, dataTagId, unitId) VALUES (:standardTag, :description, :dataTagId, :unitId) ON DUPLICATE KEY UPDATE description=:description, dataTagId=:dataTagId, unitId=:unitId;')
# with pool.connect() as db_conn:
#     # insert into database
#     for record in metaData:
#         db_conn.execute(insertMetaDataStatement, parameters=record)
#     db_conn.commit()

2023-03-12 18:44:40,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 18:44:40,881 INFO sqlalchemy.engine.Engine INSERT INTO META_DATA (standardTag, description, dataTagId, unitId) VALUES (%(standardTag)s, %(description)s, %(dataTagId)s, %(unitId)s) ON DUPLICATE KEY UPDATE description=%(description)s, dataTagId=%(dataTagId)s, unitId=%(unitId)s;
2023-03-12 18:44:40,882 INFO sqlalchemy.engine.Engine [generated in 0.00153s] {'standardTag': 'PROSS1_INN_STEAM_PRSS1', 'description': 'Process Inlet Steam Pressure', 'dataTagId': 'PT2_7_SCALE_PV', 'unitId': 2}
2023-03-12 18:44:41,137 INFO sqlalchemy.engine.Engine INSERT INTO META_DATA (standardTag, description, dataTagId, unitId) VALUES (%(standardTag)s, %(description)s, %(dataTagId)s, %(unitId)s) ON DUPLICATE KEY UPDATE description=%(description)s, dataTagId=%(dataTagId)s, unitId=%(unitId)s;
2023-03-12 18:44:41,138 INFO sqlalchemy.engine.Engine [cached since 0.2582s ago] {'standardTag': 'FURNC1_INN_FUELS_TOTZ1', 'description': 'Fur

In [172]:
mystr = 'INSERT INTO FIXED_DATA (standardTag, description, dataTagId, unitId) VALUES (:standardTag, :description, :dataTagId, :unitId) ON DUPLICATE KEY UPDATE description=:description, dataTagId=:dataTagId, unitId=:unitId;'

print(mystr.replace('dataTagId', 'value'))

INSERT INTO FIXED_DATA (standardTag, description, value, unitId) VALUES (:standardTag, :description, :value, :unitId) ON DUPLICATE KEY UPDATE description=:description, value=:value, unitId=:unitId;


In [173]:
# insertFixedDataStatement = sqlalchemy.text('INSERT INTO FIXED_DATA (standardTag, description, value, unitId) VALUES (:standardTag, :description, :value, :unitId) ON DUPLICATE KEY UPDATE description=:description, value=:value, unitId=:unitId;')
# with pool.connect() as db_conn:
#     # insert into database
#     for record in fixedData:
#         db_conn.execute(insertFixedDataStatement, parameters=record)
#     db_conn.commit()

2023-03-12 19:37:46,363 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-12 19:37:46,364 INFO sqlalchemy.engine.Engine INSERT INTO FIXED_DATA (standardTag, description, value, unitId) VALUES (%(standardTag)s, %(description)s, %(value)s, %(unitId)s) ON DUPLICATE KEY UPDATE description=%(description)s, value=%(value)s, unitId=%(unitId)s;
2023-03-12 19:37:46,365 INFO sqlalchemy.engine.Engine [generated in 0.00120s] {'standardTag': 'DREFF1_MID_RLAVG_TIME1', 'description': 'Direct Efficiency Averaging Time Window', 'value': 60.0, 'unitId': 8}
2023-03-12 19:37:46,623 INFO sqlalchemy.engine.Engine INSERT INTO FIXED_DATA (standardTag, description, value, unitId) VALUES (%(standardTag)s, %(description)s, %(value)s, %(unitId)s) ON DUPLICATE KEY UPDATE description=%(description)s, value=%(value)s, unitId=%(unitId)s;
2023-03-12 19:37:46,624 INFO sqlalchemy.engine.Engine [cached since 0.2606s ago] {'standardTag': 'FURNC1_INN_FUELS_HGCV1', 'description': 'Gross calorific value of fuel in use',